In [ ]:
! pip install llama-parse

### 1. LLalmaParser 사용하기

In [1]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
from dotenv import load_dotenv # 환경 변수 설정
load_dotenv()

True

In [2]:
# 비동기 처리 문제 해결
import nest_asyncio
nest_asyncio.apply()

In [3]:
# LlamaParse 파서 설정 ("markdown"과 "text" 형식 중 선택 가능)
parser = LlamaParse(result_type="markdown")

# SimpleDirectoryReader를 사용하여 파일 파싱
file_extractor = {".pdf": parser}  # PDF 파일을 LlamaParse로 처리하도록 설정
documents = SimpleDirectoryReader(
    input_files=["./data/240510_카카오_AI투자,눈에 밟히는 비용_유진투자증권.pdf"], file_extractor=file_extractor
).load_data()


Started parsing the file under job_id 2ae7d35e-6635-4afc-9028-2c13082a8fe8
.....

In [4]:
print(documents[0])

Doc ID: d8641b6b-ed13-4d29-9236-8ebdb8046982
Text: # Korea| 인터넷 | 2024.05.10  # 카카오  # 투자의견  BUY(유지)  # (035720)
목표주가  73,000 원(유지)  # 현재주가  48,600 원(05/09)  # 시가총액  21,639(십억원)  #
인터넷/게임/우주 정의훈_ 02)368-6170_uihoon0607@eugenefn.com  - 1분기 매출액 1조
9,884억원(+22.5%yoy), 영업이익 1,203억원(+92.2%yoy), 컨센서스(1,271억원) 소폭 하회 - 톡비즈
광고 매출은 메시지 비즈니스와 비즈보드에서 모두 두 자릿수 성장률 기록하며 성장. 중국 이커머스 플랫폼의 국내 마케팅 확장
또한 동사 광고 성장에...


In [5]:
file_name = "parse_kakao_doc1.md"
with open(file_name, 'w', encoding='utf-8') as file:
    for n, doc in enumerate(documents, start=1):
        file.write(f'#####{n} page 요약\n')
        file.write(doc.text+'\n')

In [13]:

'''
document 생성시 instruction promprt 사용하기
'''
documents_with_instruction = LlamaParse(
    result_type='markdown',
    parsing_instruction="""
    전체 내용을 투자자 관점에서 간결하게 설명해주시기 바랍니다.
    문서 안에 표가 포함 된 경우 수치를 포함해 요약해주세요.
    """
).load_data("./data/240510_카카오_AI투자,눈에 밟히는 비용_유진투자증권.pdf")

Started parsing the file under job_id af7f4731-0cf3-4d0c-8d7c-d907df7b87db
.

In [14]:
file_name = "parse_kakao_doc2.md"
with open(file_name, 'w', encoding='utf-8') as file:
    for n, doc in enumerate(documents_with_instruction, start=1):
        file.write(f'#####{n} page 요약\n')
        file.write(doc.text+'\n')
        

### 2. LlamaParser를 활용한 RAG 시스템 구축

In [27]:
# OpenAI GPT-4 모델을 사용하도록 설정
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4o", max_tokens=1000)

In [ ]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents=documents_with_instruction,
                                        llm=llm)
index.storage_context.persist(persist_dir="./index/ch05_2_llamaparser_vector_index_storage") # index 저장

In [28]:
### 저장 된 인덱스 불러오기
from llama_index.core import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./index/ch05_2_llamaparser_vector_index_storage") 
index = load_index_from_storage(storage_context) # 밀집 검색 인덱스 로드

In [39]:
dense_retriever = index.as_retriever(
        similarity_top_k = 3,
)


In [40]:
retrieval_results = dense_retriever.retrieve("카카오 2024년 실적 요약")

In [43]:
from llama_index.core.response.notebook_utils import display_source_node
for res_node in retrieval_results:
    display_source_node(res_node, source_length=200)

**Node ID:** 642ae671-75c3-4521-a5f1-ef8cbdf06736<br>**Similarity:** 0.8831013999784525<br>**Text:** 카카오는 2024년 1분기 실적에서 매출 1조 9,884억 원(+22.5% yoy)과 영업이익 1,203억 원(+92.2% yoy)을 기록했으나, 영업이익은 시장 예상치인 1,271억 원과 당사 추정치 1,301억 원을 소폭 하회했습니다.

주요 성장 요인은 톡비즈 광고와 페이 부문으로, 특히 금융과 커머스 업종의 광고 활성화가 긍정적인 영향을 미쳤습...<br>

**Node ID:** 9b9ef1df-b70c-4e3f-a0a6-c80e64bfa699<br>**Similarity:** 0.8611001950555083<br>**Text:** ### 카카오 실적 요약

#### 1. 분기 실적 (2023년 1Q - 2024년 4Q 예상)
- **매출액**:
- 2023년: 7,557억 원
- 2024년 예상: 8,443억 원
- 연평균 성장률: 11.1% (2023), 11.7% (2024)

- **영업이익**:
- 2023년: 460.9억 원
- 2024년 예상: 581.8억 원
- 영...<br>

**Node ID:** 580cb1d7-8fa6-49c3-b67e-798305fbe0e6<br>**Similarity:** 0.8376004129186496<br>**Text:** ### 카카오 투자 요약

- **투자의견**: BUY (유지)
- **목표주가**: 73,000 원 (유지)
- **현재주가**: 48,600 원 (2024.05.09)
- **시가총액**: 21,639 억원

#### 1분기 실적
- **매출액**: 1조 9,884억원 (+22.5% YoY)
- **영업이익**: 1,203억원 (+92.2% YoY...<br>

In [44]:
query_engine = index.as_query_engine()

In [45]:
response = query_engine.query("카카오 2024년 실적 요약")

In [46]:
print(response)

카카오의 2024년 실적 요약은 다음과 같습니다:
- 매출액: 8,443억 원
- 영업이익: 581.8억 원
- 당기순이익: 448.5억 원
- EPS: 1,008원
- PER: 48.2배
